In [3]:
import pickle
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
import numpy as np

In [4]:
path_df = "Data Analysis/Questions_dataset.pickle"

with open(path_df, 'rb') as data:
    df = pickle.load(data)

In [5]:
df.head()

,Questions,Category,Questions_length
0,About what proportion of the population of the...,BT1,69
1,Correctly label the brain lobes indicated on t...,BT1,62
2,Define compound interest.,BT1,25
3,Define four types of traceability,BT1,33
4,Define mercantilism.,BT1,20


In [8]:
df.loc[1]['Questions']

'Correctly label the brain lobes indicated on the diagram below'

In [9]:
# \r and \n
df['Questions_Parsed_1'] = df['Questions'].str.replace("\r", " ")
df['Questions_Parsed_1'] = df['Questions_Parsed_1'].str.replace("\n", " ")
df['Questions_Parsed_1'] = df['Questions_Parsed_1'].str.replace("    ", " ")

In [11]:
text = "Mr Vijay\'s"
text

"Mr Vijay's"

In [12]:
# " when quoting text
df['Questions_Parsed_1'] = df['Questions_Parsed_1'].str.replace('"', '')

In [13]:

# Lowercasing the text
df['Questions_Parsed_2'] = df['Questions_Parsed_1'].str.lower()

In [14]:
punctuation_signs = list("?:!.,;")
df['Questions_Parsed_3'] = df['Questions_Parsed_2']

for punct_sign in punctuation_signs:
    df['Questions_Parsed_3'] = df['Questions_Parsed_3'].str.replace(punct_sign, '')

In [15]:
df['Questions_Parsed_4'] = df['Questions_Parsed_3'].str.replace("'s", "")

In [16]:
# Downloading punkt and wordnet from NLTK
nltk.download('punkt')
print("------------------------------------------------------------")
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\acer\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


------------------------------------------------------------


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\acer\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [17]:
# Saving the lemmatizer into an object
wordnet_lemmatizer = WordNetLemmatizer()

In [18]:
nrows = len(df)
lemmatized_text_list = []

for row in range(0, nrows):
    
    # Create an empty list containing lemmatized words
    lemmatized_list = []
    
    # Save the text and its words into an object
    text = df.loc[row]['Questions_Parsed_4']
    text_words = text.split(" ")

    # Iterate through every word to lemmatize
    for word in text_words:
        lemmatized_list.append(wordnet_lemmatizer.lemmatize(word, pos="v"))
        
    # Join the list
    lemmatized_text = " ".join(lemmatized_list)
    
    # Append to the list containing the texts
    lemmatized_text_list.append(lemmatized_text)

In [19]:
df['Questions_Parsed_5'] = lemmatized_text_list

In [20]:
# Downloading the stop words list
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\acer\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [21]:
# Loading the stop words in english
stop_words = list(stopwords.words('english'))

In [22]:
stop_words[0:15]

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours']

In [23]:
example = "me name is vijay"
word = "me"

# The regular expression is:
regex = r"\b" + word + r"\b"  # we need to build it like that to work properly

re.sub(regex, "StopWord", example)

'StopWord name is vijay'

In [24]:
df['Questions_Parsed_6'] = df['Questions_Parsed_5']

for stop_word in stop_words:

    regex_stopword = r"\b" + stop_word + r"\b"
    df['Questions_Parsed_6'] = df['Questions_Parsed_6'].str.replace(regex_stopword, '')

In [31]:
df.loc[445]['Questions']

'What happened after Shahajans death'

In [32]:
df.loc[445]['Questions_Parsed_1']

'What happened after Shahajans death'

In [33]:
df.loc[445]['Questions_Parsed_2']

'what happened after shahajans death'

In [35]:
df.loc[445]['Questions_Parsed_3']

'what happened after shahajans death'

In [36]:
df.loc[445]['Questions_Parsed_4']

'what happened after shahajans death'

In [37]:
df.loc[445]['Questions_Parsed_5']

'what happen after shahajans death'

In [38]:
df.loc[445]['Questions_Parsed_6']

' happen  shahajans death'

In [39]:
df.head(1)

,Questions,Category,Questions_length,Questions_Parsed_1,Questions_Parsed_2,Questions_Parsed_3,Questions_Parsed_4,Questions_Parsed_5,Questions_Parsed_6
0,About what proportion of the population of the...,BT1,69,About what proportion of the population of the...,about what proportion of the population of the...,about what proportion of the population of the...,about what proportion of the population of the...,about what proportion of the population of the...,proportion population us live farm


In [40]:
list_columns = ["Category", "Questions", "Questions_Parsed_6"]
df = df[list_columns]

df = df.rename(columns={'Questions_Parsed_6': 'Questions_Parsed'})

In [41]:
df.head()

,Category,Questions,Questions_Parsed
0,BT1,About what proportion of the population of the...,proportion population us live farm
1,BT1,Correctly label the brain lobes indicated on t...,correctly label brain lob indicate diagram
2,BT1,Define compound interest.,define compound interest
3,BT1,Define four types of traceability,define four type traceability
4,BT1,Define mercantilism.,define mercantilism


In [42]:
category_codes = {
    'BT1': 0,
    'BT2': 1,
    'BT3': 2,
    'BT4': 3,
    'BT5': 4,
    'BT6': 5
}

In [43]:
# Category mapping
df['Category_Code'] = df['Category']
df = df.replace({'Category_Code':category_codes})

In [44]:
df.head()

,Category,Questions,Questions_Parsed,Category_Code
0,BT1,About what proportion of the population of the...,proportion population us live farm,0
1,BT1,Correctly label the brain lobes indicated on t...,correctly label brain lob indicate diagram,0
2,BT1,Define compound interest.,define compound interest,0
3,BT1,Define four types of traceability,define four type traceability,0
4,BT1,Define mercantilism.,define mercantilism,0


In [46]:
X_train, X_test, y_train, y_test = train_test_split(df['Questions_Parsed'], 
                                                    df['Category_Code'], 
                                                    test_size=0.15, 
                                                    random_state=8)

In [47]:
# Parameter election
ngram_range = (1,2)
min_df = 10
max_df = 1.
max_features = 300

In [48]:
tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        max_features=max_features,
                        norm='l2',
                        sublinear_tf=True)
                        
features_train = tfidf.fit_transform(X_train).toarray()
labels_train = y_train
print(features_train.shape)

features_test = tfidf.transform(X_test).toarray()
labels_test = y_test
print(features_test.shape)

(1058, 219)
(187, 219)


In [49]:
from sklearn.feature_selection import chi2
import numpy as np

for Product, category_id in sorted(category_codes.items()):
    features_chi2 = chi2(features_train, labels_train == category_id)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidf.get_feature_names())[indices]
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
    print("# '{}' category:".format(Product))
    print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-5:])))
    print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-2:])))
    print("")

# 'BT1' category:
  . Most correlated unigrams:
. label
. many
. list
. define
. name
  . Most correlated bigrams:
. show figure
. show fig

# 'BT2' category:
  . Most correlated unigrams:
. classify
. discuss
. describe
. word
. explain
  . Most correlated bigrams:
. circuit diagram
. show figure

# 'BT3' category:
  . Most correlated unigrams:
. figure
. apply
. calculate
. circuit
. show
  . Most correlated bigrams:
. circuit show
. show figure

# 'BT4' category:
  . Most correlated unigrams:
. differentiate
. analyze
. compare
. distinguish
. contrast
  . Most correlated bigrams:
. control system
. show fig

# 'BT5' category:
  . Most correlated unigrams:
. character
. book
. develop
. new
. create
  . Most correlated bigrams:
. show figure
. show fig

# 'BT6' category:
  . Most correlated unigrams:
. position
. best
. evaluate
. decide
. think
  . Most correlated bigrams:
. show figure
. show fig



In [50]:
bigrams

['circuit show',
 'circuit diagram',
 'output voltage',
 'control system',
 'show figure',
 'show fig']

In [52]:
# X_train
with open('Feature Engineering/Pickles/X_train.pickle', 'wb') as output:
    pickle.dump(X_train, output)
    
# X_test    
with open('Feature Engineering/Pickles/X_test.pickle', 'wb') as output:
    pickle.dump(X_test, output)
    
# y_train
with open('Feature Engineering/Pickles/y_train.pickle', 'wb') as output:
    pickle.dump(y_train, output)
    
# y_test
with open('Feature Engineering/Pickles/y_test.pickle', 'wb') as output:
    pickle.dump(y_test, output)
    
# df
with open('Feature Engineering/Pickles/df.pickle', 'wb') as output:
    pickle.dump(df, output)
    
# features_train
with open('Feature Engineering/Pickles/features_train.pickle', 'wb') as output:
    pickle.dump(features_train, output)

# labels_train
with open('Feature Engineering/Pickles/labels_train.pickle', 'wb') as output:
    pickle.dump(labels_train, output)

# features_test
with open('Feature Engineering/Pickles/features_test.pickle', 'wb') as output:
    pickle.dump(features_test, output)

# labels_test
with open('Feature Engineering/Pickles/labels_test.pickle', 'wb') as output:
    pickle.dump(labels_test, output)
    
# TF-IDF object
with open('Feature Engineering/Pickles/tfidf.pickle', 'wb') as output:
    pickle.dump(tfidf, output)